In [ ]:
# -*- coding: utf-8 -*-
import tkinter as tk
from tkinter import ttk, filedialog, messagebox, scrolledtext
import pandas as pd
import threading
import time
import re
from bs4 import BeautifulSoup
import ollama # Ensure ollama >= 0.2.0 installed
# import json # No se usa activamente, se puede quitar si no se planea usar
from datetime import datetime
import logging
import os
import traceback
import concurrent.futures # Para procesamiento paralelo
from typing import Optional, Tuple, List, Dict, Any # Para Type Hinting

# --- Constantes Globales ---
APP_TITLE: str = "Analizador y Clasificador Optimizado de Tickets IT"
WINDOW_SIZE: str = "750x600" # Ligeramente más grande para el botón Cancelar
REQUIRED_COLUMNS: List[str] = [
    'Number', 'Priority', 'State', 'Assigned to', 'Short description',
    'Task type', 'Subcategory', 'Closed', 'Created', 'Assignment group',
    'Work notes list', 'Work notes', 'Description'
]
LLM_MODEL_GEMMA: str = 'gemma3' # Nombre base del modelo LLM
MAX_RETRIES: int = 3
RETRY_DELAY: int = 5
# ¡NUEVO! Número máximo de hilos para procesar tickets en paralelo
# Ajusta según tu CPU y la capacidad de respuesta de tu servidor Ollama
MAX_WORKERS: int = 4

# --- Constantes para Marcadores y Errores ---
EMPTY_TEXT_MARKER: str = "TEXTO_VACIO"
PROCESSING_ERROR_MARKER: str = "ERROR_PROCESAMIENTO_INTERNO"
OLLAMA_ERROR_PREFIX: str = "ERROR_OLLAMA_"
UNEXPECTED_ERROR_PREFIX: str = "ERROR_INESPERADO_"
LENGTH_MISMATCH_MARKER: str = "ERROR_LONGITUD_RESULTADOS"

# --- Plantillas de Prompts para el LLM ---
# (Sin cambios respecto a la versión anterior, asegúrate que son las correctas)
PROMPT_GEMMA_SUMMARY: str = """
Eres un asistente experto en análisis de tickets de IT. Resume el siguiente ticket de forma concisa para análisis de tendencias futuras. Enfócate en:
- Puntos clave técnicos del problema o solicitud.
- Posible causa raíz si es identificable.
- Acciones técnicas realizadas si se mencionan.

Formato de salida: Lista de viñetas (- Punto 1). Sé breve y directo.

Ticket:
{ticket_text}

Resumen conciso:
"""

PROMPT_GEMMA_CLASSIFICATION: str = """
Eres un analista de soporte IT experimentado. Lee el siguiente ticket y clasifícalo en UNA categoría principal concisa que describa el área o tipo de problema/solicitud. Sé lo más específico posible dentro de categorías comunes de IT.
Algunos ejemplos de categorías (puedes usar otras si son más apropiadas):
Gestión de Cuentas, Problema Hardware (PC/Laptop), Problema Hardware (Impresora), Problema Hardware (Otro), Problema Software (Aplicación X), Problema Software (Office), Problema Software (Sistema Operativo), Solicitud Software, Red/Conectividad, Correo Electrónico, Seguridad, Acceso VPN, Impresión, Consulta General, Capacitación, Otros.

Ticket:
{ticket_text}

Categoría Principal:
"""

# --- Configuración del Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(threadName)s - %(message)s')

# --- Funciones Auxiliares ---

def clean_text(text: Optional[Any]) -> str:
    """
    Limpia el texto de entrada (maneja None, NaN). Elimina HTML, texto específico,
    y espacios/líneas en blanco excesivos.
    """
    if not isinstance(text, str):
        return "" # Devuelve cadena vacía si no es string (o es NaN, None)
    # Elimina líneas específicas (puedes añadir más patrones si es necesario)
    text = re.sub(r'\[ARGONAUTA\].*?\n', '', text, flags=re.IGNORECASE | re.DOTALL)
    text = re.sub(r'\[INC\w*\]', '', text, flags=re.IGNORECASE)
    # Elimina HTML
    try:
        soup = BeautifulSoup(text, "html.parser")
        text = soup.get_text(separator="\n")
    except Exception as e:
        logging.warning(f"BeautifulSoup falló al parsear texto. Error: {e}. Devolviendo texto semi-limpio.")
    # Normaliza saltos de línea y espacios
    text = re.sub(r'\s*\n\s*', '\n', text) # Líneas en blanco con espacios
    text = re.sub(r'\n{3,}', '\n\n', text) # Máximo dos saltos de línea seguidos
    return text.strip()

def safe_get(dct: Dict, *keys: str) -> Optional[Any]:
    """Accede de forma segura a claves anidadas en un diccionario."""
    for key in keys:
        try:
            dct = dct[key]
        except (KeyError, TypeError, IndexError):
            return None
    return dct

def call_ollama_with_retry(model_name: str, prompt: str, max_retries: int = MAX_RETRIES, delay: int = RETRY_DELAY) -> str:
    """
    Llama a Ollama con reintentos. Devuelve la respuesta o un string de error.
    """
    retries = 0
    while retries < max_retries:
        # ¡Importante! Verificar cancelación antes de la llamada
        if app and app.cancel_requested.is_set(): # Verifica si 'app' existe
             return "OPERACION_CANCELADA"

        try:
            response = ollama.chat(model=model_name, messages=[{'role': 'user', 'content': prompt}])
            content = safe_get(response, 'message', 'content')

            if content is None:
                logging.warning(f"Respuesta inesperada Ollama ({model_name}), falta 'message.content'. Respuesta: {response}")
                raise ValueError(f"Respuesta inesperada Ollama: falta 'message.content'") # Forza reintento
            if not isinstance(content, str):
                 logging.warning(f"Respuesta Ollama no es string ({model_name}). Contenido: '{content}'")
                 raise ValueError(f"Respuesta Ollama no es string") # Forza reintento

            return content.strip()

        except (ollama.ResponseError, ConnectionError, TimeoutError, ValueError) as e:
            retries += 1
            logging.warning(f"Error Ollama ({model_name}) - Intento {retries}/{max_retries}: {e}")
            if app and app.cancel_requested.is_set(): return "OPERACION_CANCELADA"
            if retries < max_retries:
                time.sleep(delay)
            else:
                logging.error(f"Fallaron {max_retries} intentos con Ollama ({model_name}). Error final: {e}")
                return f"{OLLAMA_ERROR_PREFIX}{model_name.replace(':', '_').upper()}"
        except Exception as e:
             retries += 1
             logging.warning(f"Error inesperado llamando a Ollama ({model_name}) - Intento {retries}/{max_retries}: {e}\n{traceback.format_exc()}")
             if app and app.cancel_requested.is_set(): return "OPERACION_CANCELADA"
             if retries < max_retries:
                 time.sleep(delay)
             else:
                 logging.error(f"Fallaron {max_retries} intentos con Ollama ({model_name}) error inesperado. Error final: {e}")
                 return f"{UNEXPECTED_ERROR_PREFIX}{model_name.replace(':', '_').upper()}"
    # En teoría, nunca se llega aquí, pero por si acaso:
    return f"{UNEXPECTED_ERROR_PREFIX}FALLO_REINTENTOS_INESPERADO"


def check_model_exists(model_base_name: str, available_models_full_names: List[str]) -> Tuple[bool, Optional[str]]:
    """Verifica si un modelo base (o variante con tag) existe en la lista."""
    for full_name in available_models_full_names:
        if full_name == model_base_name or full_name.startswith(model_base_name + ':'):
            return True, full_name
    return False, None

# --- Clase de la Aplicación GUI ---
class TicketAnalyzerApp:
    def __init__(self, root: tk.Tk):
        self.root = root
        self.root.title(APP_TITLE)

        # Configuración ventana
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        w, h = map(int, WINDOW_SIZE.split('x'))
        x = (screen_width - w) // 2
        y = (screen_height - h) // 2
        self.root.geometry(f'{w}x{h}+{x}+{y}')
        self.root.minsize(w, h)

        # Variables de estado Tkinter y de control
        self.file_path = tk.StringVar()
        self.processing_thread: Optional[threading.Thread] = None
        self.start_time: Optional[float] = None
        self.gemma_summary_errors: int = 0
        self.gemma_classification_errors: int = 0
        self.processed_count: int = 0
        self.total_tickets_to_process: int = 0
        self.actual_model_to_use: str = LLM_MODEL_GEMMA
        self.cancel_requested = threading.Event() # Evento para la cancelación

        # Configuración Grid Layout
        self.root.grid_columnconfigure(0, weight=1)
        self.root.grid_rowconfigure(1, weight=1) # Fila del log expandible

        # Crear Widgets
        self._create_widgets()

        # Iniciar verificación de Ollama en hilo separado
        self._log_message("Iniciando verificación de conexión con Ollama en segundo plano...")
        threading.Thread(target=self._perform_initial_ollama_check_async, daemon=True).start()

    # --- Métodos para la GUI (Logging, Progreso, Estado UI) ---

    def _log_message(self, message: str, level: str = "INFO") -> None:
        """Registra mensajes en GUI y logger (seguro para hilos)."""
        timestamp = datetime.now().strftime("%H:%M:%S")
        log_entry = f"[{timestamp}] {message}\n"
        try:
            # Solo el hilo principal de Tkinter puede actualizar la GUI
            self.root.after(0, self._insert_log, log_entry)
            if level == "ERROR": logging.error(message)
            elif level == "WARNING": logging.warning(message)
            else: logging.info(message)
        except tk.TclError: # Si la ventana se cierra mientras se intenta loguear
            print(log_entry.strip())
        except RuntimeError: # Si el event loop de Tkinter ya no corre
             print(f"(Tkinter no disponible) {log_entry.strip()}")


    def _insert_log(self, log_entry: str) -> None:
        """Inserta texto en el widget ScrolledText (llamado desde _log_message)."""
        if self.log_text and self.log_text.winfo_exists():
            current_state = self.log_text['state']
            self.log_text.config(state=tk.NORMAL)
            self.log_text.insert(tk.END, log_entry)
            self.log_text.see(tk.END) # Auto-scroll
            self.log_text.config(state=current_state) # Restaurar estado original (usualmente DISABLED)
            # self.root.update_idletasks() # Quitado: puede ralentizar si hay muchos logs

    def _update_progress(self, value: float) -> None:
        """Actualiza la barra de progreso (seguro para hilos)."""
        try:
            self.root.after(0, self._set_progress, value)
        except (tk.TclError, RuntimeError):
            pass # Ignorar si la GUI no está disponible

    def _set_progress(self, value: float) -> None:
        """Establece el valor de la barra de progreso."""
        if self.progress_bar and self.progress_bar.winfo_exists():
            self.progress_bar['value'] = value
            # self.root.update_idletasks() # Quitado: puede ralentizar

    def _set_ui_state(self, state: str) -> None:
        """Habilita/deshabilita controles de la GUI."""
        tk_state = tk.NORMAL if state == 'enabled' else tk.DISABLED
        entry_state = tk.NORMAL if state == 'enabled' else 'readonly'
        run_button_text = "Ejecutar Proceso (▶)"
        run_button_state = tk.DISABLED

        if state == 'processing':
            tk_state = tk.DISABLED
            entry_state = 'readonly'
            run_button_text = "Procesando..."
            run_button_state = tk.DISABLED
            cancel_button_state = tk.NORMAL # Habilitar Cancelar
        elif state == 'enabled':
            cancel_button_state = tk.DISABLED # Deshabilitar Cancelar
            # Habilitar Ejecutar solo si hay archivo
            if self.file_path.get():
                run_button_state = tk.NORMAL
        else: # disabled
            cancel_button_state = tk.DISABLED

        try:
            # Usar `winfo_exists()` por si la ventana se cierra durante el proceso
            if self.browse_button and self.browse_button.winfo_exists():
                self.browse_button.config(state=tk_state)
            if self.file_entry and self.file_entry.winfo_exists():
                self.file_entry.config(state=entry_state)
            if self.run_button and self.run_button.winfo_exists():
                self.run_button.config(state=run_button_state, text=run_button_text)
            if self.cancel_button and self.cancel_button.winfo_exists():
                self.cancel_button.config(state=cancel_button_state)
        except (tk.TclError, RuntimeError):
            pass # Ignorar si la GUI no está disponible

    # --- Creación de Widgets ---
    def _create_widgets(self) -> None:
        """Crea y organiza los widgets de la GUI."""
        # --- Marco Superior (Archivo) ---
        top_frame = ttk.Frame(self.root, padding="10")
        top_frame.grid(row=0, column=0, columnspan=2, sticky="ew")
        top_frame.grid_columnconfigure(1, weight=1)
        ttk.Label(top_frame, text="Archivo Excel:").grid(row=0, column=0, padx=(0, 5), sticky="w")
        self.file_entry = ttk.Entry(top_frame, textvariable=self.file_path, state="readonly", width=60)
        self.file_entry.grid(row=0, column=1, padx=5, sticky="ew")
        self.browse_button = ttk.Button(top_frame, text="Examinar (🖿)", command=self._browse_file, width=15)
        self.browse_button.grid(row=0, column=2, padx=(5, 0))

        # --- Marco Medio (Log y Progreso) ---
        middle_frame = ttk.Frame(self.root, padding=(10, 0, 10, 10))
        middle_frame.grid(row=1, column=0, columnspan=2, sticky="nsew")
        middle_frame.grid_rowconfigure(0, weight=1)
        middle_frame.grid_columnconfigure(0, weight=1)
        # Log
        log_container = ttk.LabelFrame(middle_frame, text="Registro de Proceso", padding=5)
        log_container.grid(row=0, column=0, sticky="nsew", pady=(0, 10))
        log_container.grid_rowconfigure(0, weight=1)
        log_container.grid_columnconfigure(0, weight=1)
        self.log_text = scrolledtext.ScrolledText(log_container, wrap=tk.WORD, state=tk.DISABLED, height=15, font=("TkDefaultFont", 9))
        self.log_text.grid(row=0, column=0, sticky="nsew")
        # Barra de progreso
        self.progress_bar = ttk.Progressbar(middle_frame, orient="horizontal", length=300, mode="determinate")
        self.progress_bar.grid(row=1, column=0, sticky="ew", pady=(5, 0))

        # --- Marco Inferior (Botones) ---
        bottom_frame = ttk.Frame(self.root, padding="10")
        # Ahora la fila 2, columna 0, pero necesita columnspan=2 si quieres centrar botones
        bottom_frame.grid(row=2, column=0, columnspan=2, sticky="ew")
        # Centrar los botones en el espacio disponible
        bottom_frame.grid_columnconfigure(0, weight=1) # Espacio a la izquierda
        bottom_frame.grid_columnconfigure(1, weight=0) # Botón Ejecutar
        bottom_frame.grid_columnconfigure(2, weight=0) # Botón Cancelar
        bottom_frame.grid_columnconfigure(3, weight=1) # Espacio a la derecha

        self.run_button = ttk.Button(bottom_frame, text="Ejecutar Proceso (▶)", command=self._start_processing, state=tk.DISABLED)
        self.run_button.grid(row=0, column=1, padx=5) # Columna 1

        self.cancel_button = ttk.Button(bottom_frame, text="Cancelar (⏹)", command=self._request_cancel, state=tk.DISABLED)
        self.cancel_button.grid(row=0, column=2, padx=5) # Columna 2

    # --- Lógica de Verificación de Ollama ---

    def _verify_ollama_model(self) -> Tuple[bool, str, Optional[str]]:
        """
        Verifica conexión con Ollama y existencia del modelo base.
        Returns:
            Tuple[bool, str, Optional[str]]: (éxito, mensaje, nombre_modelo_encontrado)
        """
        try:
            models_response = ollama.list()
            # Extraer nombres de modelos de forma robusta
            available_models_full = []
            models_list = []
            if isinstance(models_response, dict) and 'models' in models_response:
                models_list = models_response['models']
            elif hasattr(models_response, 'models'): # Si es un objeto con atributo 'models'
                 models_list = models_response.models

            if isinstance(models_list, list):
                 for m in models_list:
                    model_name_attr = None
                    if isinstance(m, dict) and 'name' in m: model_name_attr = m['name'] # Estructura común
                    elif isinstance(m, dict) and 'model' in m: model_name_attr = m['model'] # Otra estructura posible
                    elif hasattr(m, 'model'): model_name_attr = m.model
                    elif hasattr(m, 'name'): model_name_attr = m.name

                    if model_name_attr: available_models_full.append(model_name_attr)
                    else: logging.warning(f"Elemento de modelo inesperado en lista Ollama: {m}")
            else:
                 return False, f"Respuesta inesperada de Ollama al listar modelos: {type(models_list)}", None

            model_found, found_model_name = check_model_exists(LLM_MODEL_GEMMA, available_models_full)

            if model_found:
                return True, f"Modelo base '{LLM_MODEL_GEMMA}' encontrado (se usará: '{found_model_name}').", found_model_name
            else:
                return False, f"Modelo base '{LLM_MODEL_GEMMA}' (o variante) NO encontrado. Instálalo (ej: 'ollama pull {LLM_MODEL_GEMMA}').", None

        except (ollama.ResponseError, ConnectionError, TimeoutError) as e:
            return False, f"Error de conexión/respuesta con Ollama: {e}. ¿Está Ollama ejecutándose?", None
        except Exception as e:
            return False, f"Error inesperado verificando Ollama: {e}\n{traceback.format_exc()}", None

    def _perform_initial_ollama_check_async(self) -> None:
        """Realiza la verificación inicial en un hilo separado."""
        is_ok, message, found_model_name = self._verify_ollama_model()
        log_level = "INFO" if is_ok else "ERROR"
        self._log_message(f"Verificación inicial Ollama: {message}", level=log_level)
        if is_ok and found_model_name:
            self.actual_model_to_use = found_model_name
        elif not is_ok:
             # Opcional: Mostrar un popup de error no bloqueante si falla al inicio
             # self.root.after(0, lambda: messagebox.showerror("Error Ollama Inicial", message, parent=self.root))
             pass # Por ahora solo log

    # --- Lógica de Selección de Archivo ---
    def _browse_file(self) -> None:
        """Abre diálogo para seleccionar archivo Excel."""
        fpath = filedialog.askopenfilename(
            parent=self.root,
            title="Seleccionar archivo Excel",
            filetypes=[("Archivos Excel", "*.xlsx"), ("Todos los archivos", "*.*")]
        )
        if fpath:
            if not fpath.lower().endswith(".xlsx"):
                messagebox.showerror("Error de Archivo", "Selecciona un archivo .xlsx válido.", parent=self.root)
                self.file_path.set("")
            else:
                self.file_path.set(fpath)
                # Limpiar estado anterior
                if self.log_text.winfo_exists():
                    self.log_text.config(state=tk.NORMAL)
                    self.log_text.delete('1.0', tk.END)
                    self.log_text.config(state=tk.DISABLED)
                self._update_progress(0)
                self.gemma_summary_errors = 0
                self.gemma_classification_errors = 0
                self.processed_count = 0
                self.total_tickets_to_process = 0
                self._log_message(f"Archivo seleccionado: {fpath}")
            # Actualizar estado de botones (habilita Ejecutar si hay ruta)
            self._set_ui_state('enabled')
        else:
             # Si canceló y no había ruta previa, asegurar que Ejecutar esté deshabilitado
             if not self.file_path.get():
                 self._set_ui_state('disabled') # Llama a set_ui_state para manejar la lógica

    # --- Lógica Principal de Procesamiento ---

    def _start_processing(self) -> None:
        """Valida e inicia el proceso de análisis en un hilo."""
        if not self.file_path.get():
            messagebox.showwarning("Archivo no seleccionado", "Selecciona un archivo Excel.", parent=self.root)
            return
        if self.processing_thread and self.processing_thread.is_alive():
            messagebox.showwarning("Proceso en ejecución", "Análisis ya en curso.", parent=self.root)
            return

        # --- Verificación Crítica Pre-vuelo ---
        is_ok, message, found_model_name = self._verify_ollama_model()
        if not is_ok:
            self._log_message(f"ERROR CRÍTICO (Pre-vuelo): {message}", level="ERROR")
            messagebox.showerror("Error Crítico Ollama", f"No se puede iniciar el proceso.\n{message}", parent=self.root)
            return
        else:
            self._log_message(f"Verificación pre-vuelo OK. Modelo a usar: {found_model_name}")
            self.actual_model_to_use = found_model_name # Asegurarse de usar el nombre completo

        # --- Preparar e Iniciar Hilo ---
        self._set_ui_state('processing') # Cambia estado a procesando (habilita Cancelar)
        self._update_progress(0)
        self.gemma_summary_errors = 0
        self.gemma_classification_errors = 0
        self.processed_count = 0
        self.total_tickets_to_process = 0
        self.start_time = time.time()
        self.cancel_requested.clear() # Asegura que no esté cancelado de ejecuciones previas

        # Limpiar log
        if self.log_text.winfo_exists():
            self.log_text.config(state=tk.NORMAL); self.log_text.delete('1.0', tk.END); self.log_text.config(state=tk.DISABLED)

        self._log_message(f"Iniciando análisis paralelo (hasta {MAX_WORKERS} hilos)...")
        self._log_message(f"Usando modelo LLM: {self.actual_model_to_use}")

        # Iniciar hilo worker
        self.processing_thread = threading.Thread(target=self._process_tickets_worker, name="WorkerThread", daemon=True)
        self.processing_thread.start()

    def _request_cancel(self) -> None:
        """Señaliza la solicitud de cancelación."""
        if self.processing_thread and self.processing_thread.is_alive():
            self._log_message("Solicitud de cancelación recibida...", level="WARNING")
            self.cancel_requested.set()
            if self.cancel_button and self.cancel_button.winfo_exists():
                 self.cancel_button.config(state=tk.DISABLED) # Deshabilitar tras pulsar
        else:
            self._log_message("No hay proceso activo para cancelar.", level="INFO")

    def _process_single_ticket(self, ticket_data: Any) -> Tuple[int, str, str, str]:
        """
        Procesa un único ticket (limpieza, llamadas LLM).
        Se ejecuta en un hilo del ThreadPoolExecutor.
        Args:
            ticket_data: Un objeto NamedTuple de df.itertuples() que representa una fila.
        Returns:
            Tuple[int, str, str, str]: (índice_original, número_ticket, resultado_resumen, resultado_clasificación)
        """
        original_index = ticket_data.Index
        ticket_number = str(getattr(ticket_data, 'Number', f'Fila_{original_index + 1}'))

        # Verificar cancelación al inicio de la tarea
        if self.cancel_requested.is_set():
            return original_index, ticket_number, "OPERACION_CANCELADA", "OPERACION_CANCELADA"

        try:
            # --- Combinar y Limpiar Texto ---
            short_desc = str(getattr(ticket_data, 'Short_description', '')) # Ojo: itertuples puede renombrar columnas con espacios
            description = str(getattr(ticket_data, 'Description', ''))
            work_notes = str(getattr(ticket_data, 'Work_notes', ''))
            # Revisa los nombres exactos que genera itertuples si fallan los getattr
            # Puedes imprimir `dir(ticket_data)` la primera vez para verlos.
            # Si tienen caracteres especiales, usa getattr(ticket_data, 'Nombre Columna')

            combined_text = f"Título: {short_desc}\n\nDescripción:\n{description}\n\nNotas de trabajo:\n{work_notes}"
            cleaned_text = clean_text(combined_text)

            # --- Procesar si hay texto ---
            if not cleaned_text.strip():
                logging.warning(f"T:{ticket_number} - Texto vacío tras limpieza.")
                return original_index, ticket_number, EMPTY_TEXT_MARKER, EMPTY_TEXT_MARKER

            # Verificar cancelación antes de llamadas LLM
            if self.cancel_requested.is_set(): return original_index, ticket_number, "OPERACION_CANCELADA", "OPERACION_CANCELADA"

            # --- 1. Generar Resumen ---
            summary_prompt = PROMPT_GEMMA_SUMMARY.format(ticket_text=cleaned_text)
            gemma_summary = call_ollama_with_retry(self.actual_model_to_use, summary_prompt)

            if self.cancel_requested.is_set(): return original_index, ticket_number, "OPERACION_CANCELADA", "OPERACION_CANCELADA"

             # --- 2. Generar Clasificación ---
            classification_prompt = PROMPT_GEMMA_CLASSIFICATION.format(ticket_text=cleaned_text)
            gemma_classification = call_ollama_with_retry(self.actual_model_to_use, classification_prompt)

            # Limpieza básica de clasificación (si no es error ni cancelado)
            if not gemma_classification.startswith("ERROR_") and gemma_classification != "OPERACION_CANCELADA":
                gemma_classification = gemma_classification.lstrip("-* ").strip()

            return original_index, ticket_number, gemma_summary, gemma_classification

        except Exception as e:
            # Capturar cualquier error inesperado DENTRO del procesamiento de UN ticket
            logging.error(f"Error inesperado procesando T:{ticket_number} (Índice: {original_index}): {e}\n{traceback.format_exc()}")
            return original_index, ticket_number, PROCESSING_ERROR_MARKER, PROCESSING_ERROR_MARKER


    def _process_tickets_worker(self) -> None:
        """Worker principal: lee Excel, gestiona pool de hilos, guarda resultados."""
        df = None
        summaries_map: Dict[int, str] = {} # Usar diccionarios para mapear índice a resultado
        classifications_map: Dict[int, str] = {}

        try:
            file_path = self.file_path.get()
            self._log_message(f"Leyendo archivo: {os.path.basename(file_path)}", level="INFO")

            # --- Lectura y Validación ---
            try:
                df = pd.read_excel(file_path, engine='openpyxl')
            # (Manejo de errores de lectura como antes)
            except FileNotFoundError:
                self._log_message(f"Error: Archivo no encontrado: {file_path}", level="ERROR"); self.root.after(0, lambda: messagebox.showerror("Error", f"Archivo no encontrado:\n{file_path}", parent=self.root)); self._finalize_processing(); return
            except Exception as e:
                self._log_message(f"Error al leer Excel: {e}\n{traceback.format_exc()}", level="ERROR"); self.root.after(0, lambda: messagebox.showerror("Error Lectura", f"No se pudo leer Excel:\n{e}", parent=self.root)); self._finalize_processing(); return

            # Validar columnas (¡importante!)
            # Limpiar nombres de columnas del DF para comparación robusta
            df.columns = [col.strip() for col in df.columns]
            missing_cols = [col for col in REQUIRED_COLUMNS if col not in df.columns]
            if missing_cols:
                missing_cols_str = ', '.join(missing_cols)
                self._log_message(f"Error: Faltan columnas: {missing_cols_str}", level="ERROR")
                self.root.after(0, lambda: messagebox.showerror("Error Columnas", f"Faltan columnas requeridas:\n{missing_cols_str}", parent=self.root))
                self._finalize_processing(); return

            self.total_tickets_to_process = len(df)
            if self.total_tickets_to_process == 0:
                self._log_message("Archivo Excel vacío.", level="WARNING"); self.root.after(0, lambda: messagebox.showwarning("Archivo Vacío", "Excel no contiene tickets.", parent=self.root)); self._finalize_processing(); return

            # Configurar máximo de la barra de progreso
            self.root.after(0, lambda: self.progress_bar.config(maximum=self.total_tickets_to_process))
            self._log_message(f"Validación OK. Total tickets a procesar: {self.total_tickets_to_process}")

            # --- Procesamiento Paralelo ---
            with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS, thread_name_prefix='TicketProcessor') as executor:
                # Crear futuros (tareas) para cada fila del DataFrame usando itertuples
                # Guardar el futuro asociado a su índice original para recuperarlo después
                # ¡Ojo! itertuples puede cambiar nombres de columnas con espacios o caracteres especiales
                # Revisa los nombres si getattr falla en _process_single_ticket
                futures: Dict[concurrent.futures.Future, int] = {
                    executor.submit(self._process_single_ticket, ticket_row): ticket_row.Index
                    for ticket_row in df.itertuples(index=True, name='TicketRow') # 'name' es opcional
                }

                for future in concurrent.futures.as_completed(futures):
                    original_index = futures[future] # Recuperar índice original
                    try:
                        # Obtener resultado de la tarea completada
                        idx, t_num, summary_res, classification_res = future.result()

                        # Almacenar resultados en los diccionarios usando el índice original
                        summaries_map[original_index] = summary_res
                        classifications_map[original_index] = classification_res

                        # Contar errores específicos de Ollama o procesamiento interno
                        if str(summary_res).startswith(OLLAMA_ERROR_PREFIX) or str(summary_res).startswith(UNEXPECTED_ERROR_PREFIX) or summary_res == PROCESSING_ERROR_MARKER:
                            self.gemma_summary_errors += 1
                            self._log_message(f"Error resumen T:{t_num} (Índice:{idx}): {summary_res}", level="ERROR")
                        if str(classification_res).startswith(OLLAMA_ERROR_PREFIX) or str(classification_res).startswith(UNEXPECTED_ERROR_PREFIX) or classification_res == PROCESSING_ERROR_MARKER:
                            self.gemma_classification_errors += 1
                            self._log_message(f"Error clasificación T:{t_num} (Índice:{idx}): {classification_res}", level="ERROR")
                        elif summary_res == "OPERACION_CANCELADA":
                             # No contar como error, pero se puede loguear si se desea
                             pass

                        self.processed_count += 1
                        # Actualizar progreso y log de forma segura
                        self._update_progress(self.processed_count)
                        if self.processed_count % 10 == 0 or self.processed_count == self.total_tickets_to_process: # Log menos frecuente
                             self._log_message(f"Progreso: {self.processed_count}/{self.total_tickets_to_process} tickets procesados.")

                    except Exception as exc:
                        # Error al obtener el resultado de un futuro (raro, pero posible)
                        self.processed_count += 1 # Contar como intentado
                        logging.error(f"Error al obtener resultado del futuro para índice {original_index}: {exc}", exc_info=True)
                        # Guardar marcadores de error para este ticket
                        summaries_map[original_index] = f"ERROR_FUTURO: {exc}"
                        classifications_map[original_index] = f"ERROR_FUTURO: {exc}"
                        self.gemma_summary_errors += 1
                        self.gemma_classification_errors += 1
                        self._update_progress(self.processed_count)


                    # --- Comprobar Cancelación ---
                    # Importante comprobar *después* de procesar cada futuro
                    if self.cancel_requested.is_set():
                        self._log_message("Cancelación detectada en worker. Deteniendo envío de nuevas tareas y esperando activas...", level="WARNING")
                        # Intentar cancelar futuros pendientes (Requiere Python 3.9+)
                        # for f in futures:
                        #     if not f.done(): f.cancel()
                        # Shutdown non-blocking, permite que las tareas en ejecución terminen si no se pueden cancelar
                        executor.shutdown(wait=False) # No esperar aquí si queremos cancelar rápido
                        break # Salir del bucle as_completed

            # --- Fin Procesamiento Paralelo ---

            if self.cancel_requested.is_set():
                 self._log_message("Proceso cancelado por el usuario.", level="WARNING")
                 # No guardar archivo si se canceló (o decidir si guardar parcial)
                 self._finalize_processing(cancelled=True)
                 return # Salir del worker

            # --- Ensamblar Resultados y Guardar ---
            self._log_message("Ensamblando resultados...")
            if df is not None:
                # Crear las listas de resultados ORDENADAS según el índice original del DataFrame
                # Es crucial para que los resultados coincidan con las filas correctas
                ordered_summaries = [summaries_map.get(i, LENGTH_MISMATCH_MARKER) for i in df.index]
                ordered_classifications = [classifications_map.get(i, LENGTH_MISMATCH_MARKER) for i in df.index]

                # Verificar si la longitud coincide (doble chequeo)
                if len(ordered_summaries) != len(df) or len(ordered_classifications) != len(df):
                     self._log_message(f"¡Advertencia Crítica! Discrepancia de longitud final.", level="ERROR")
                     # Manejar este caso extremo si ocurre (poco probable con este método)

                # Añadir columnas al DataFrame
                model_name_base = self.actual_model_to_use.split(':')[0].capitalize()
                output_col_summary = f"Resumen_{model_name_base}"
                output_col_classification = f"Clasificacion_{model_name_base}"
                df[output_col_summary] = ordered_summaries
                df[output_col_classification] = ordered_classifications
                self._log_message("Resultados añadidos al DataFrame.")

                # --- Guardar Output ---
                # (Lógica de guardado como antes, usando ask_save_path en hilo principal)
                self._log_message("Solicitando ubicación para guardar...")
                save_path_container = {}; save_event = threading.Event()
                def ask_save_path():
                    try:
                        initial_filename = os.path.basename(file_path); name, ext = os.path.splitext(initial_filename)
                        suggested_filename=f"{name}_analizado_clasificado_{model_name_base.lower()}{ext}"
                        s_path = filedialog.asksaveasfilename(parent=self.root, title="Guardar archivo analizado", defaultextension=".xlsx", initialfile=suggested_filename, filetypes=[("Excel files", "*.xlsx")])
                        save_path_container['path'] = s_path
                    except Exception as e: self._log_message(f"Error diálogo guardado: {e}", level="ERROR"); save_path_container['path'] = None
                    finally: save_event.set()
                self.root.after(0, ask_save_path); save_event.wait()
                save_path = save_path_container.get('path')

                if save_path:
                    try:
                        self._log_message(f"Guardando archivo en: {save_path}")
                        df.to_excel(save_path, index=False, engine='openpyxl')
                        self._log_message(f"Archivo guardado exitosamente: {save_path}")
                        self.root.after(0, lambda: messagebox.showinfo("Completado", f"Guardado en:\n{save_path}", parent=self.root))
                    except Exception as e:
                        self._log_message(f"Error al guardar Excel: {e}\n{traceback.format_exc()}", level="ERROR"); self.root.after(0, lambda: messagebox.showerror("Error Guardar", f"No se pudo guardar Excel:\n{e}", parent=self.root))
                else:
                    self._log_message("Guardado cancelado por el usuario.", level="WARNING"); self.root.after(0, lambda: messagebox.showwarning("Cancelado", "Archivo no guardado.", parent=self.root))
            else:
                self._log_message("Error: No hay DataFrame para guardar.", level="ERROR")

        except Exception as e: # Catch-all para worker thread
            self._log_message(f"Error GRABE inesperado en worker: {e}\n{traceback.format_exc()}", level="ERROR")
            self.root.after(0, lambda: messagebox.showerror("Error Inesperado", f"Error procesamiento:\n{e}", parent=self.root))
        finally:
            # Llamar a finalizar, indicando si fue cancelado o no
            was_cancelled = self.cancel_requested.is_set()
            self._finalize_processing(cancelled=was_cancelled)

    def _finalize_processing(self, cancelled: bool = False) -> None:
        """Limpia UI y loguea estadísticas finales."""
        end_time = time.time()
        total_time = end_time - (self.start_time if self.start_time else end_time)
        model_name_used = self.actual_model_to_use

        self._log_message("="*30)
        if cancelled:
             self._log_message("--- PROCESO CANCELADO ---", level="WARNING")
        else:
             self._log_message("--- PROCESO COMPLETADO ---")
        self._log_message("Estadísticas Finales:")
        self._log_message(f"Tiempo total: {total_time:.2f} seg")
        self._log_message(f"Tickets totales en archivo: {self.total_tickets_to_process}")
        self._log_message(f"Tickets procesados (intentados): {self.processed_count}")
        self._log_message(f"Errores Resumen ({model_name_used}): {self.gemma_summary_errors}")
        self._log_message(f"Errores Clasificación ({model_name_used}): {self.gemma_classification_errors}")

        # Actualizar UI al estado final 'enabled' (en hilo principal)
        self.root.after(0, self._set_ui_state, 'enabled')
        # Asegurar que progreso refleje el final (incluso si canceló)
        self._update_progress(self.processed_count)

        # Resetear para próxima ejecución
        self.processing_thread = None
        self.start_time = None
        # No limpiar cancel_requested aquí, se limpia al iniciar el *próximo* proceso

# --- Ejecución Principal ---
if __name__ == "__main__":
    root = tk.Tk()
    # Aplicar tema (opcional pero mejora apariencia)
    try:
        style = ttk.Style(root)
        available_themes = style.theme_names()
        for theme in ['clam', 'alt', 'default', 'vista', 'xpnative']: # Probar varios comunes
            if theme in available_themes:
                style.theme_use(theme)
                break
    except Exception as e:
        print(f"No se pudo aplicar tema ttk: {e}")

    # Crear y lanzar la aplicación
    # Hacer 'app' global o pasarla a funciones que necesiten acceso al estado (como call_ollama)
    app = TicketAnalyzerApp(root)
    root.mainloop()